# Convert ASCII Grid File to Cloud Optimised GeoTIFF

Downloads a zipped ASCII grid elevation file and exports it as a COG.

The zip file contains:
1. The raster data (.asc file)
2. The data's projection (.prj file)
3. A metadata html file (which we'll ignore)


### Load packages

In [2]:
# import boto3
import io
import os
import pathlib
import rasterio.crs
import requests
import zipfile

from rasterio.io import MemoryFile
from rio_cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles


### Settings

In [3]:
# the directory of this script
script_dir = os.path.dirname(os.path.realpath(__file__))

# the URL of the elevation data to download
url = "https://portal.spatial.nsw.gov.au/download/dem/56/Sydney-DEM-AHD_56_5m.zip"

# the local path or AWS S3 path to save the images to
output_path = os.path.join(script_dir, "data")

### Download file

Download a ZIP file and extract its contents in-memory.

Yields a list of (filename, file-like object) pairs

In [4]:
%%time

file_list = list()

# download zipfile
response = requests.get(url)

# extract each file from zipfile
with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
    for zipinfo in thezip.infolist():
        with thezip.open(zipinfo) as thefile:
            file_list.append((zipinfo.filename, thefile.read()))            

Eastern-DEM-AHD_56_5m_Metadata.html
CPU times: user 3.26 s, sys: 597 ms, total: 3.86 s
Wall time: 7.89 s


In [ ]:
### Load raster file & get it's coordinate system

Also, save raster file to disk for testing (not required for conversion to COG)

In [ ]:
image = None
crs = None
output_file_name = None
    
for file in file_list:
    file_name = file[0]
    file_obj = file[1]

    # get raster as an in-memory file
    if file_name.endswith(".asc"):
        image = MemoryFile(file_obj)
        output_file_name = file_name.replace(".asc", ".tif")

    # get well known text coordinate system
    if file_name.endswith(".prj"):
        proj_string = file_obj.decode("utf-8")
        crs = rasterio.crs.CRS.from_wkt(proj_string)

    if debug:
    with open(os.path.join(output_path, file_name), "wb") as f:
        f.write(file_obj)


In [ ]:
# set Sydney map bounds (in lat/long)

x_min = 150.45
x_max = 151.45
y_min = -34.15
y_max = y_min + (map_height / map_width) * (x_max - x_min)

bounds = dict(x_range = (x_min, x_max), y_range = (y_min, y_max))

# plot the points
cvs = datashader.Canvas(plot_width=map_width, plot_height=map_height, **bounds)
agg = cvs.points(df, 'x', 'y', datashader.count())

tf.set_background(tf.shade(agg.where(agg > 0), cmap=colorcet.fire), "black")

In [ ]:
# set Inner Sydney map bounds (in lat/long)

x_min = 150.8
x_max = 151.3
y_min = -34.0
y_max = y_min + (map_height / map_width) * (x_max - x_min)

bounds = dict(x_range = (x_min, x_max), y_range = (y_min, y_max))

# plot the points
cvs = datashader.Canvas(plot_width=map_width, plot_height=map_height, **bounds)
agg = cvs.points(df, 'x', 'y', datashader.count())

tf.set_background(tf.shade(agg.where(agg > 0), cmap=colorcet.blues), "white")

In [ ]:
sizes  = dict(width=plot_width, height=int(plot_width*0.5))
opts   = dict(xaxis=None, yaxis=None, bgcolor="black", **sizes)
points = hv.Points(df, ['x', 'y'])
dynspread(datashade(points, cmap=["red", "yellow"], **sizes)).options(**opts)